# Hakai CTD Profile QA/QC Development Tool
This jupyter notebook was developed to provide a flexible tool for testing and improving Hakai's QA/QC workflow for CTD profile data. 

The tool loads Hakai's CTD dataset and allows the user to modify the default tests applied to the data so that different QA/QC thresholds can be evaluated. In addition, new tests can be developed and applied to the data to test thier effectiveness. 

## Load packages
The initial load of these packages may take considerable time. 

Some of the packages needed are already available on the google Colab Server (pandas, seaborn, matplotlib, json plotly). Others need to be downloaded and installed from their repository on the Hakai and IOOS Github account with the Pypi(pip) package: 
- [hakai-api-client-python](https://github.com/HakaiInstitute/hakai-api-client-python)
- [ioos_qc](https://github.com/HakaiInstitute/ioos_qc@colab-compatible)(Hakai Institute Fork and colab-compatible branch) ioos_qc is actively in developpement. To minimize issues with new changes to the ioos branch we are using a forked version on the Hakai Domain.
- [hakai_qc](https://github.com/HakaiInstitute/hakai-profile-qaqc) This package is the main package developped to run the different tests on the Hakai CTD profile dataset. 
- ipywidgets which makes possible the interactive plotting tool!

In [1]:
# If runned on Google Colab install the different packages needed
!pip install git+https://github.com/HakaiInstitute/hakai-api-client-python.git
!pip install git+https://github.com/HakaiInstitute/ioos_qc@colab-compatible
!pip install git+https://github.com/HakaiInstitute/hakai-profile-qaqc.git
!pip install ipywidgets

  Running command git clone -q https://github.com/HakaiInstitute/hakai-api-client-python.git 'C:\Users\jessy\AppData\Local\Temp\pip-req-build-lepcalzj'



  Cloning https://github.com/HakaiInstitute/hakai-api-client-python.git to c:\users\jessy\appdata\local\temp\pip-req-build-lepcalzj
  Created wheel for hakai-api: filename=hakai_api-0.2.0-py3-none-any.whl size=2833 sha256=5e93543d31d01d901e0bd4d7f44dc45fe866f48ca8d5ecc5ce550dad7257d9b3
  Stored in directory: C:\Users\jessy\AppData\Local\Temp\pip-ephem-wheel-cache-6jcob8os\wheels\46\ef\c2\23d927e6aec7699f34f8ae70af6c6bd505d5895d04060d7cb2
Successfully built hakai-api
  Cloning https://github.com/HakaiInstitute/ioos_qc (to revision colab-compatible) to c:\users\jessy\appdata\local\temp\pip-req-build-o4q29hau

  Running command git clone -q https://github.com/HakaiInstitute/ioos_qc 'C:\Users\jessy\AppData\Local\Temp\pip-req-build-o4q29hau'
  Running command git checkout -b colab-compatible --track origin/colab-compatible
  Branch 'colab-compatible' set up to track remote branch 'colab-compatible' from 'origin'.
  Switched to a new branch 'colab-compatible'



  Created wheel for ioos-qc: filename=ioos_qc-1.0.0-py3-none-any.whl size=37544 sha256=2c97ffd5ca2f7bf30fcb8c32811cadda18d62b517cc6ac6c7c5d75bd14f4a33e
  Stored in directory: C:\Users\jessy\AppData\Local\Temp\pip-ephem-wheel-cache-3ss8d76e\wheels\9f\a9\a1\3ad9f78862dc108d8ea432bb53b5fcba20cfa72bc651e25b27
Successfully built ioos-qc
  Cloning https://github.com/HakaiInstitute/hakai-profile-qaqc.git to c:\users\jessy\appdata\local\temp\pip-req-build-0vn847z1


  Running command git clone -q https://github.com/HakaiInstitute/hakai-profile-qaqc.git 'C:\Users\jessy\AppData\Local\Temp\pip-req-build-0vn847z1'


  Created wheel for hakai-qc: filename=hakai_qc-0.0.1-py3-none-any.whl size=39052 sha256=7cb494bd4dc331582015df5dbe90e5ca97b0397bfdbe512d3b79e4c3817a4ec6
  Stored in directory: C:\Users\jessy\AppData\Local\Temp\pip-ephem-wheel-cache-oj33rs01\wheels\c1\0b\c2\b92dfc44c2cb46c91be29fc6310031a4a38273888654dfe242
Successfully built hakai-qc


In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px


# Get External packages
from hakai_api import Client
from ioos_qc.config import QcConfig
import hakai_qc
import ipywidgets as widgets
from ipywidgets import interact
    
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


## Import Stations and CTD data
Below, Hakai oceanography stations are uploaded. The Hakai Station Master List is based on a CSV output of the [Hakai Oceanography Master Stations Map and Data](https://hakai.maps.arcgis.com/apps/webappviewer/index.html?id=38e1b1da8d16466bbe5d7c7a713d2678). In order to have the tests applied, new/missing stations will need to be added to the master list.

In [4]:
# Load Hakai Station List
hakai_stations = hakai_qc.get.hakai_stations()

The following code allows the user to select a station of interest (i.e. QU39) and then, using the API, downloads the processed CTD data for this station from the Hakai database. Note that the API query will need to be authorized, which requires following the link that appears below the code block (appears after the code block has been run) and copying/pasting the authentication URL into the provided box. The paste dropdown button does not work here, so use CNTRL-V to paste the URL into the box.

In [6]:
# Get Hakai CTD Data Download through the API
station = 'QU39'

variable_lists = hakai_qc.get.hakai_api_selected_variables()

# Let's just get the data from QU39
filterUrl = 'station='+station+'&status!=MISCAST&limit=-1'+'&fields='+','.join(variable_lists)
df, url = hakai_qc.get.hakai_ctd_data(filterUrl)
print(str(len(df))+' records found')

# Regroup profiles and sort them by pressure
group_variables = ['device_model','device_sn','ctd_file_pk','ctd_cast_pk','direction_flag']
df = df.sort_values(by=group_variables+['pressure'])

# Get Derived Variables
df = hakai_qc.utils.derived_ocean_variables(df)

# Just show the first few lines to have a look
df.head() # Show the top of the data frame

Please go here and authorize:
https://hecate.hakai.org/api/auth/oauth2?response_type=code&client_id=289782143400-1f4r7l823cqg8fthd31ch4ug0thpejme.apps.googleusercontent.com&state=e6FYNQwwIhEuYQjp26L4RbgllNK6C2

Paste the full redirect URL here:
https://hecate.hakai.org/api/auth/oauth2/callback?state=e6FYNQwwIhEuYQjp26L4RbgllNK6C2&code=4/0AY0e-g4e-zFz-Z1NnmThGJXNmdKm-zfsFye8fw8yhCnw4AInb7Srjh3PPtyByDeY3b49_g&scope=email%20profile%20openid%20https://www.googleapis.com/auth/userinfo.email%20https://www.googleapis.com/auth/userinfo.profile&authuser=0&hd=hakai.org&prompt=none
138793 records found


C:\Users\jessy\Anaconda3\lib\site-packages\gsw\_wrapped_ufuncs.py:696: RuntimeWarning:

invalid value encountered in ct_from_t



,ctd_file_pk,ctd_cast_pk,hakai_id,ctd_data_pk,filename,device_model,device_sn,work_area,cruise,station,...,sos_un,sos_un_flag,backscatter_beta,backscatter_beta_flag,cdom_ppb,cdom_ppb_flag,absolute salinity,conservative temperature,density,sigma0
137812,6626,16083,065679_2021-02-08T17:33:33.667Z,10377321,065679_20210208_1043,RBRconcerto,065679,QUADRA,BIOSIEGE,QU39,...,None,None,None,None,None,None,24.821185,6.303582,1019.414797,19.410121
137813,6626,16083,065679_2021-02-08T17:33:33.667Z,10377322,065679_20210208_1043,RBRconcerto,065679,QUADRA,BIOSIEGE,QU39,...,None,None,None,None,None,None,27.718701,6.813937,1021.630228,21.620951
137814,6626,16083,065679_2021-02-08T17:33:33.667Z,10377323,065679_20210208_1043,RBRconcerto,065679,QUADRA,BIOSIEGE,QU39,...,None,None,None,None,None,None,27.824868,7.187401,1021.673417,21.659532
137815,6626,16083,065679_2021-02-08T17:33:33.667Z,10377324,065679_20210208_1043,RBRconcerto,065679,QUADRA,BIOSIEGE,QU39,...,None,None,None,None,None,None,28.025992,7.403323,1021.808505,21.790020
137816,6626,16083,065679_2021-02-08T17:33:33.667Z,10377325,065679_20210208_1043,RBRconcerto,065679,QUADRA,BIOSIEGE,QU39,...,None,None,None,None,None,None,28.046546,7.444634,1021.824018,21.800917


## Hakai CTD Profile QA/QC Test Configuration

This step imports and configures (i.e sets test thresholds) the QA/QC tests that will be applied to the Hakai CTD data uploaded from the database. 

New tests can be added by following the structure within the code block below. For additional information on the different tests available, please look at the [ioos_qc webpage](https://ioos.github.io/ioos_qc/api/ioos_qc.html).

Test thresholds can be modified using the structure provided in the fluoroescence example below. Variable abbreviations, required to change the thresholds, will appear in a table once this code block has been run.


In [9]:
# Load default test parameters used right now!
qc_config = hakai_qc.get.json_config('hakai_ctd_profile.json')

#Test parameters can be modified below following the provided format for fluorescence 
#ex: This code sets the range outside of which fluorescence is considered suspect and implausible (fail) 
qc_config['flc']= {'qartod': {
                        'gross_range_test': {   
                            "suspect_span": [0, 70],
                            "fail_span": [-.5, 100],
                        }
                   }}

# Displays the QC/QA test parameters in a table
pd.set_option('display.max_rows', 100)
hakai_qc.get.config_as_dataframe(qc_config)

Value
Variable                 Module Test                   Input                                                               
position                 qartod location_test          bbox                                            [-180, -90, 180, 90]
                                                       target_range                                                    3000
pressure                 qartod gross_range_test       fail_span                                                 [0, 12000]
                                                       maximum_fail_depth_ratio                                         1.1
                                                       maximum_suspect_depth_ratio                                     1.05
                                                       suspect_span                                              [0, 12000]
depth                    qartod gross_range_test       fail_span                                                 [0, 12000]
                                                       maximum_fail_depth_ratio                                         1.1
                                                       maximum_suspect_depth_ratio                                     1.05
                                                       suspect_span                                              [0, 12000]
dissolved_oxygen_ml_l    qartod gross_range_test       fail_span                                                    [0, 20]
                                                       suspect_span                                                 [1, 15]
                                rate_of_change_test    threshold                                                          3
                                spike_test             suspect_threshold                                                0.5
                                                       fail_threshold                                                     1
                                attenuated_signal_test suspect_threshold                                                0.1
                                                       fail_threshold                                                  0.01
                                                       check_type                                                     range
dissolved_oxygen_percent qartod gross_range_test       fail_span                                                  [-1, 150]
                                                       suspect_span                                                [0, 140]
                                rate_of_change_test    threshold                                                         30
                                spike_test             suspect_threshold                                                 20
                                                       fail_threshold                                                    40
                                attenuated_signal_test suspect_threshold                                                0.1
                                                       fail_threshold                                                  0.01
                                                       check_type                                                     range
rinko_do_ml_l            qartod gross_range_test       fail_span                                                    [0, 20]
                                                       suspect_span                                                 [1, 15]
                                rate_of_change_test    threshold                                                          3
                                spike_test             suspect_threshold                                                0.5
                                                       fail_threshold                                                     1
                          

## Apply QA/QC Tests to CTD Profiles

The QA/QC tests and thresholds defined in the above code block/table are applied to the CTD data on a profile by profile basis. Only profiles from the pre-selected station will be evaluated. 

In [10]:
# Run all of the tests on each available profile
df = hakai_qc.run.tests_on_profiles(df,hakai_stations,qc_config)

QAQC QU39
  position
    qartod
      ('location_test', {'bbox': [-180, -90, 180, 90], 'target_range': 3000, 'target_lat': [50.0307000000001], 'target_lon': [-125.0992]})
  pressure
    qartod
      ('gross_range_test', {'suspect_span': [0, 280.83826311944904], 'fail_span': [0, 294.2115137441847], 'maximum_suspect_depth_ratio': 1.05, 'maximum_fail_depth_ratio': 1.1})
  depth
    qartod
      ('gross_range_test', {'suspect_span': [0, 278.25], 'fail_span': [0, 291.5], 'maximum_suspect_depth_ratio': 1.05, 'maximum_fail_depth_ratio': 1.1})
  dissolved_oxygen_ml_l
    qartod
      ('gross_range_test', {'fail_span': [0, 20], 'suspect_span': [1, 15]})
      ('rate_of_change_test', {'threshold': 3})
      ('spike_test', {'suspect_threshold': 0.5, 'fail_threshold': 1})
      ('attenuated_signal_test', {'suspect_threshold': 0.1, 'fail_threshold': 0.01, 'check_type': 'range'})
  dissolved_oxygen_percent
    qartod
      ('gross_range_test', {'fail_span': [-1, 150], 'suspect_span': [0, 140]})
    

No ioos_qc package "hakai" was found, skipping.


  hakai_tests
    hakai
      ('do_cap_test', {'variable': ['dissolved_oxygen_ml_l', 'rinko_do_ml_l'], 'bin_size': 1, 'suspect_threshold': 0.2, 'fail_threshold': 0.5, 'ratio_above_threshold': 0.5, 'minimum_bins_per_profile': 10})
      ('bottom_hit_detection', {'variable': 'sigma0_qartod_density_inversion_test', 'profile_direction_variable': 'direction_flag'})
      ('par_shadow_test', {'variable': 'par', 'min_par_for_shadow_detection': 5})
      ('bad_value_test', {'variable': 'all', 'flag_list': ['.isna', -9.99e-29]})


KeyError: 'hakai_tests'

## Review Results
###  Profile location versus target location
This section present the CTD profile qartod location_test result which is comparing the CTD drop location versus the target station coordinates as defined in the Hakai Station Master list. We present here all the drops that failed the test:

In [ ]:
# Output all drops with flagged positions that exceed the distance threshold or do not display coordinates (NaN)
#  ignore rows where a depth value does not exist.
df[df['position_qartod_location_test']>1].dropna(
    axis=0,subset=['depth']).groupby(
    'hakai_id').first()[['position_qartod_location_test','station','latitude','longitude','measurement_dt']]

In [ ]:
# Display the CTD profiles with flagged GPS coordinates
m = hakai_qc.show.flag_result_map(df.dropna(axis=0,subset=['latitude','longitude','depth']),
                                  flag_variable='position_qartod_location_test')
m

### Review Profile test flags
We present here an interactive way of plotting and reviewing the different flags. Just run the following cell and interact with the figure below.

#### First let's review statistically the results of the tests!

In [ ]:
hakai_qc.show.test_results_bar_plot(df,test_list= sorted(df.filter(regex='_flag$|_test$').columns.tolist()))

#### Review one profile at the time

In [ ]:
variable_list = set(qc_config.keys())-{'position'}
test_list = sorted(df.filter(regex='_flag$|_test$').columns.tolist())
test_list.append("None")

hakai_qc.show.interactive_profile_viewer(df,variable_list,test_list)

#### Plot data as scatter time series

In [ ]:
# Just find quickly max range of pressure and select a range to plot around the midddle of 
#  the wate column to start with
var = 'dissolved_oxygen_ml_l'
pressure = [120,200]

# Depth scatter data
df_plot = df[(df['direction_flag']=='d')& (df[var+'_qartod_flag']==1) & (df[var].notna())]
fig = px.scatter(df_plot,x='measurement_dt',y='depth',color=var,hover_data=['hakai_id'])
fig.update_yaxes(autorange="reversed")
fig.show()

# Plot time series
df_plot_depth = df_plot[(df_plot['pressure']>pressure[0]) & (df_plot['pressure']<pressure[1])]
fig2 = px.scatter(df_plot_depth,x='measurement_dt',y=var,color='pressure')
fig2.show()

#### Or make contour plots

In [ ]:
var = 'dissolved_oxygen_ml_l'

# Plot contourf interpolate linearly over the x axis and maximum over two NaN values
df_pivot = pd.pivot_table(df_plot,values=var,index='pressure',columns='start_dt',aggfunc='mean').interpolate(axis=1,limit=2)
fig = go.Figure(data =
    go.Contour(z=df_pivot.values,x=df_pivot.columns,y=df_pivot.index.values,
               colorbar=dict(title=var, titleside='right')
               #,connectgaps=True
              ))
fig.add_trace(go.Scatter(x=df_pivot.columns,y=[0]*len(df_pivot.columns),
                         marker_symbol='triangle-down',marker_line_width=2, marker_size=10,
                         marker_line_color="black", marker_color="white",mode='markers'))

fig.update_yaxes(title='Pressure (dBar)',autorange="reversed",linecolor='black',mirror=True,
                 ticks='outside',showline=True)
fig.update_xaxes(linecolor='black',mirror=True,ticks='outside',showline=True)
fig.show()